In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import itertools
from keras.layers import Input, Dense, Reshape, Flatten
from keras import layers, initializers
from keras.models import Model, load_model
import keras.backend as K
import numpy as np
from seqtools import SequenceTools as ST
from gfp_gp import SequenceGP
from util import AA, AA_IDX
from util import build_vae
from sklearn.model_selection import train_test_split, ShuffleSplit
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
from gan import WGAN
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import scipy.stats
from scipy.stats import norm
from scipy.optimize import minimize
from keras.utils.generic_utils import get_custom_objects
from util import one_hot_encode_aa, partition_data, get_balaji_predictions, get_samples
from util import convert_idx_array_to_aas, build_pred_vae_model, get_experimental_X_y
from util import get_gfp_X_y_aa
from losses import neg_log_likelihood
import json
plt.rcParams['figure.dpi'] = 300
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

def contain_tf_gpu_mem_usage() :
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    set_session(sess)

contain_tf_gpu_mem_usage()


Using TensorFlow backend.


In [2]:

it = 0

TRAIN_SIZE = 5000
train_size_str = "%ik" % (TRAIN_SIZE/1000)
num_models = [1, 5, 20][it]
RANDOM_STATE = it + 1

X_train, y_train, gt_train  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)


In [3]:

print(np.percentile(y_train, 50))
print(np.percentile(y_train, 80))
print(np.percentile(y_train, 95))
print(np.percentile(y_train, 100))


3.12042461331375
3.142383237048391
3.1558655520498666
3.1798934456199195


In [4]:
def build_model(M):
    x = Input(shape=(M, 20,))
    y = Flatten()(x)
    y = Dense(50, activation='elu')(y)
    y = Dense(2)(y)
    model = Model(inputs=x, outputs=y)
    return model

def evaluate_ground_truth(X_aa, ground_truth, save_file=None):
    y_gt = ground_truth.predict(X_aa, print_every=100000)[:, 0]
    if save_file is not None:
        np.save(save_file, y_gt)
        
def train_and_save_oracles(X_train, y_train, n=10, suffix='', batch_size=100):
    for i in range(n):
        model = build_model(X_train.shape[1])
        model.compile(optimizer='adam',
                      loss=neg_log_likelihood,
                      )
        early_stop = EarlyStopping(monitor='val_loss', 
                                   min_delta=0, 
                                   patience=5, 
                                   verbose=1)

        model.fit(X_train, y_train, 
                  epochs=100, 
                  batch_size=batch_size, 
                  validation_split=0.1, 
                  callbacks=[early_stop],
                  verbose=2)
        model.save("models/oracle_%i%s.h5" % (i, suffix))

In [5]:
import editdistance

def compute_edit_distance(seqs, opt_len=None) :
    shuffle_index = np.arange(len(seqs))
    shuffle_index = shuffle_index[::-1]#np.random.shuffle(shuffle_index)
    
    seqs_shuffled = [seqs[shuffle_index[i]] for i in range(len(seqs))]
    edit_distances = np.ravel([float(editdistance.eval(seq_1, seq_2)) for seq_1, seq_2 in zip(seqs, seqs_shuffled)])
    if opt_len is not None :
        edit_distances /= opt_len
    
    return edit_distances

def weighted_ml_opt(X_train, oracles, ground_truth, vae_0, weights_type='dbas',
        LD=20, iters=20, samples=500, homoscedastic=False, homo_y_var=0.1,
        quantile=0.95, verbose=False, alpha=1, train_gt_evals=None,
        cutoff=1e-6, it_epochs=10, enc1_units=50):
    
    assert weights_type in ['cbas', 'dbas','rwr', 'cem-pi', 'fbvae']
    L = X_train.shape[1]
    vae = build_vae(latent_dim=LD,
                    n_tokens=20, seq_length=L,
                    enc1_units=enc1_units)

    traj = np.zeros((iters, 7))
    oracle_samples = np.zeros((iters, samples))
    gt_samples = np.zeros((iters, samples))
    edit_distance_samples = np.zeros((iters, samples))
    oracle_max_seq = None
    oracle_max = -np.inf
    gt_of_oracle_max = -np.inf
    y_star = -np.inf
    
    
    # FOR REVIEW:
    all_seqs = pd.DataFrame(0, index=range(int((iters-1)*samples)), columns=['seq', 'val'])
    l_ = 0
    
    for t in range(iters):
        ### Take Samples ###
        zt = np.random.randn(samples, LD)
        if t > 0:
            Xt_p = vae.decoder_.predict(zt)
            Xt = get_samples(Xt_p)
        else:
            Xt = X_train
        
        ### Evaluate ground truth and oracle ###
        yt, yt_var = get_balaji_predictions(oracles, Xt)
        if homoscedastic:
            yt_var = np.ones_like(yt) * homo_y_var
        Xt_aa = np.argmax(Xt, axis=-1)
        if t == 0 and train_gt_evals is not None:
            yt_gt = train_gt_evals
        else:
            yt_gt = ground_truth.predict(Xt_aa, print_every=1000000)[:, 0]
        
        ### Calculate weights for different schemes ###
        if t > 0:
            if weights_type == 'cbas': 
                log_pxt = np.sum(np.log(Xt_p) * Xt, axis=(1, 2))
                X0_p = vae_0.decoder_.predict(zt)
                log_px0 = np.sum(np.log(X0_p) * Xt, axis=(1, 2))
                w1 = np.exp(log_px0-log_pxt)
                y_star_1 = np.percentile(yt, quantile*100)
                if y_star_1 > y_star:
                    y_star = y_star_1
                w2= scipy.stats.norm.sf(y_star, loc=yt, scale=np.sqrt(yt_var))
                weights = w1*w2 
            elif weights_type == 'cem-pi':
                pi = scipy.stats.norm.sf(max_train_gt, loc=yt, scale=np.sqrt(yt_var))
                pi_thresh = np.percentile(pi, quantile*100)
                weights = (pi > pi_thresh).astype(int)
            elif weights_type == 'dbas':
                y_star_1 = np.percentile(yt, quantile*100)
                if y_star_1 > y_star:
                    y_star = y_star_1
                weights = scipy.stats.norm.sf(y_star, loc=yt, scale=np.sqrt(yt_var))
            elif weights_type == 'rwr':
                weights = np.exp(alpha*yt)
                weights /= np.sum(weights)
        else:
            weights = np.ones(yt.shape[0])
            max_train_gt = np.max(yt_gt)
            
        yt_max_idx = np.argmax(yt)
        yt_max = yt[yt_max_idx]
        if yt_max > oracle_max:
            oracle_max = yt_max
            try:
                oracle_max_seq = convert_idx_array_to_aas(Xt_aa[yt_max_idx-1:yt_max_idx])[0]
            except IndexError:
                print(Xt_aa[yt_max_idx-1:yt_max_idx])
            gt_of_oracle_max = yt_gt[yt_max_idx]
        
        ### Record and print results ##
        if t == 0:
            rand_idx = np.random.randint(0, len(yt), samples)
            oracle_samples[t, :] = yt[rand_idx]
            gt_samples[t, :] = yt_gt[rand_idx]
            edit_distance_samples[t, :] = compute_edit_distance(convert_idx_array_to_aas(Xt_aa[rand_idx, ...]))
        if t > 0:
            oracle_samples[t, :] = yt
            gt_samples[t, :] = yt_gt
            edit_distance_samples[t, :] = compute_edit_distance(convert_idx_array_to_aas(Xt_aa))
        
        traj[t, 0] = np.max(yt_gt)
        traj[t, 1] = np.mean(yt_gt)
        traj[t, 2] = np.std(yt_gt)
        traj[t, 3] = np.max(yt)
        traj[t, 4] = np.mean(yt)
        traj[t, 5] = np.std(yt)
        traj[t, 6] = np.mean(yt_var)
        
        if verbose:
            print(weights_type.upper(), t, traj[t, 0], color.BOLD + str(traj[t, 1]) + color.END, 
                  traj[t, 2], traj[t, 3], color.BOLD + str(traj[t, 4]) + color.END, traj[t, 5], traj[t, 6], np.median(edit_distance_samples[t, :]))
        
        ### Train model ###
        if t == 0:
            vae.encoder_.set_weights(vae_0.encoder_.get_weights())
            vae.decoder_.set_weights(vae_0.decoder_.get_weights())
            vae.vae_.set_weights(vae_0.vae_.get_weights())
        else:
            cutoff_idx = np.where(weights < cutoff)
            Xt = np.delete(Xt, cutoff_idx, axis=0)
            yt = np.delete(yt, cutoff_idx, axis=0)
            weights = np.delete(weights, cutoff_idx, axis=0)
            vae.fit([Xt], [Xt, np.zeros(Xt.shape[0])],
                  epochs=it_epochs,
                  batch_size=10,
                  shuffle=False,
                  sample_weight=[weights, weights],
                  verbose=0)
    
    max_dict = {'oracle_max' : oracle_max, 
                'oracle_max_seq': oracle_max_seq, 
                'gt_of_oracle_max': gt_of_oracle_max}
    return traj, oracle_samples, gt_samples, edit_distance_samples, max_dict

In [6]:
def fb_opt(X_train, oracles, ground_truth, vae_0, weights_type='fbvae',
        LD=20, iters=20, samples=500, 
        quantile=0.8, verbose=False, train_gt_evals=None,
        it_epochs=10, enc1_units=50):
    
    assert weights_type in ['fbvae']
    L = X_train.shape[1]
    vae = build_vae(latent_dim=LD,
                    n_tokens=20, seq_length=L,
                    enc1_units=enc1_units)

    traj = np.zeros((iters, 7))
    oracle_samples = np.zeros((iters, samples))
    gt_samples = np.zeros((iters, samples))
    edit_distance_samples = np.zeros((iters, samples))
    oracle_max_seq = None
    oracle_max = -np.inf
    gt_of_oracle_max = -np.inf
    y_star = - np.inf
    for t in range(iters):
        ### Take Samples and evaluate ground truth and oracle ##
        zt = np.random.randn(samples, LD)
        if t > 0:
            Xt_sample_p = vae.decoder_.predict(zt)
            Xt_sample = get_samples(Xt_sample_p)
            yt_sample, _ = get_balaji_predictions(oracles, Xt_sample)
            Xt_aa_sample = np.argmax(Xt_sample, axis=-1)
            yt_gt_sample = ground_truth.predict(Xt_aa_sample, print_every=1000000)[:, 0]
        else:
            Xt = X_train
            yt, _ = get_balaji_predictions(oracles, Xt)
            Xt_aa = np.argmax(Xt, axis=-1)
            fb_thresh = np.percentile(yt, quantile*100)
            if train_gt_evals is not None:
                yt_gt = train_gt_evals
            else:
                yt_gt = ground_truth.predict(Xt_aa, print_every=1000000)[:, 0]
        
        ### Calculate threshold ###
        if t > 0:
            threshold_idx = np.where(yt_sample >= fb_thresh)[0]
            n_top = len(threshold_idx)
            sample_arrs = [Xt_sample, yt_sample, yt_gt_sample, Xt_aa_sample]
            full_arrs = [Xt, yt, yt_gt, Xt_aa]
            
            for l in range(len(full_arrs)):
                sample_arr = sample_arrs[l]
                full_arr = full_arrs[l]
                sample_top = sample_arr[threshold_idx]
                full_arr = np.concatenate([sample_top, full_arr])
                full_arr = np.delete(full_arr, range(full_arr.shape[0]-n_top, full_arr.shape[0]), axis=0)
                full_arrs[l] = full_arr
            Xt, yt, yt_gt, Xt_aa = full_arrs
        yt_max_idx = np.argmax(yt)
        yt_max = yt[yt_max_idx]
        if yt_max > oracle_max:
            oracle_max = yt_max
            try:
                oracle_max_seq = convert_idx_array_to_aas(Xt_aa[yt_max_idx-1:yt_max_idx])[0]
            except IndexError:
                print(Xt_aa[yt_max_idx-1:yt_max_idx])
            gt_of_oracle_max = yt_gt[yt_max_idx]
        
        ### Record and print results ##

        rand_idx = np.random.randint(0, len(yt), samples)
        oracle_samples[t, :] = yt[rand_idx]
        gt_samples[t, :] = yt_gt[rand_idx]
        edit_distance_samples[t, :] = compute_edit_distance(convert_idx_array_to_aas(Xt_aa[rand_idx, ...]))

        traj[t, 0] = np.max(yt_gt)
        traj[t, 1] = np.mean(yt_gt)
        traj[t, 2] = np.std(yt_gt)
        traj[t, 3] = np.max(yt)
        traj[t, 4] = np.mean(yt)
        traj[t, 5] = np.std(yt)
        if t > 0:
            traj[t, 6] = n_top
        else:
            traj[t, 6] = 0
        
        if verbose:
            print(weights_type.upper(), t, traj[t, 0], color.BOLD + str(traj[t, 1]) + color.END, 
                  traj[t, 2], traj[t, 3], color.BOLD + str(traj[t, 4]) + color.END, traj[t, 5], traj[t, 6], np.median(edit_distance_samples[t, :]))
        
        ### Train model ###
        if t == 0:
            vae.encoder_.set_weights(vae_0.encoder_.get_weights())
            vae.decoder_.set_weights(vae_0.decoder_.get_weights())
            vae.vae_.set_weights(vae_0.vae_.get_weights())
        else:
        
            vae.fit([Xt], [Xt, np.zeros(Xt.shape[0])],
                  epochs=1,
                  batch_size=10,
                  shuffle=False,
                  verbose=0)
            
    max_dict = {'oracle_max' : oracle_max, 
                'oracle_max_seq': oracle_max_seq, 
                'gt_of_oracle_max': gt_of_oracle_max}
    return traj, oracle_samples, gt_samples, edit_distance_samples, max_dict


In [7]:
def train_experimental_oracles():
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    i = 1
    num_models = [1, 5, 20]
    for i in range(len(num_models)):
        RANDOM_STATE = i+1
        nm = num_models[i]
        X_train, y_train, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
        suffix = '_%s_%i_%i' % (train_size_str, nm, RANDOM_STATE)
        train_and_save_oracles(X_train, y_train, batch_size=10, n=nm, suffix=suffix)

In [8]:
def train_experimental_vaes(i_list=[0, 2]):
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    suffix = '_%s' % train_size_str
    for i in i_list:
        RANDOM_STATE = i + 1
        X_train, _, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
        vae_0 = build_vae(latent_dim=20,
                  n_tokens=20, 
                  seq_length=X_train.shape[1],
                  enc1_units=50)
        vae_0.fit([X_train], [X_train, np.zeros(X_train.shape[0])],
                  epochs=100,
                  batch_size=10,
                  verbose=2)
        vae_0.encoder_.save_weights("models/vae_0_encoder_weights%s_%i.h5"% (suffix, RANDOM_STATE))
        vae_0.decoder_.save_weights("models/vae_0_decoder_weights%s_%i.h5"% (suffix, RANDOM_STATE))
        vae_0.vae_.save_weights("models/vae_0_vae_weights%s_%i.h5"% (suffix, RANDOM_STATE))

In [9]:
def run_experimental_weighted_ml(it, repeat_start=0, repeats=3):
    
    assert it in [0, 1, 2]
    
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    num_models = [1, 5, 20][it]
    RANDOM_STATE = it + 1
    
    X_train, y_train, gt_train  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
    
    vae_suffix = '_%s_%i' % (train_size_str, RANDOM_STATE)
    oracle_suffix = '_%s_%i_%i' % (train_size_str, num_models, RANDOM_STATE)
    
    vae_0 = build_vae(latent_dim=20,
                  n_tokens=20, 
                  seq_length=X_train.shape[1],
                  enc1_units=50)

    vae_0.encoder_.load_weights("models/vae_0_encoder_weights%s.h5" % vae_suffix)
    vae_0.decoder_.load_weights("models/vae_0_decoder_weights%s.h5"% vae_suffix)
    vae_0.vae_.load_weights("models/vae_0_vae_weights%s.h5"% vae_suffix)
    
    ground_truth = SequenceGP(load=True, load_prefix="data/gfp_gp")
    
    loss = neg_log_likelihood
    get_custom_objects().update({"neg_log_likelihood": loss})
    #oracles = [load_model("models/oracle_%i%s.h5" % (i, oracle_suffix)) for i in range(num_models)]
    oracles = [build_model(X_train.shape[1]) for i in range(num_models)]
    for i in range(num_models) :
        oracles[i].load_weights("models/oracle_%i%s.h5" % (i, oracle_suffix))
    
    test_kwargs = [
                   {'weights_type':'cbas', 'quantile': 1},
                   {'weights_type':'rwr', 'alpha': 20},
                   {'weights_type':'dbas', 'quantile': 0.95},
                   {'weights_type':'cem-pi', 'quantile': 0.8},
                   {'weights_type': 'fbvae', 'quantile': 0.8}
    ]
    
    base_kwargs = {
        'homoscedastic': False,
        'homo_y_var': 0.01,
        'train_gt_evals':gt_train,
        'samples':100,
        'cutoff':1e-6,
        'it_epochs':10,
        'verbose':True,
        'LD': 20,
        'enc1_units':50,
        'iters': 50
    }
    
    if num_models==1:
        base_kwargs['homoscedastic'] = True
        base_kwargs['homo_y_var'] = np.mean((get_balaji_predictions(oracles, X_train)[0] - y_train)**2)
    
    for k in range(repeat_start, repeats):
        for j in range(len(test_kwargs)):
            test_name = test_kwargs[j]['weights_type']
            suffix = "_%s_%i_%i_w_edit_distances" % (train_size_str, RANDOM_STATE, k)
            if test_name == 'fbvae':
                if base_kwargs['iters'] > 100:
                    suffix += '_long'
            
                print(suffix)
                kwargs = {}
                kwargs.update(test_kwargs[j])
                kwargs.update(base_kwargs)
                [kwargs.pop(k) for k in ['homoscedastic', 'homo_y_var', 'cutoff', 'it_epochs']]
                test_traj, test_oracle_samples, test_gt_samples, test_edit_distance_samples, test_max = fb_opt(np.copy(X_train), oracles, ground_truth, vae_0, **kwargs)
            else:
                if base_kwargs['iters'] > 100:
                    suffix += '_long'
                kwargs = {}
                kwargs.update(test_kwargs[j])
                kwargs.update(base_kwargs)
                test_traj, test_oracle_samples, test_gt_samples, test_edit_distance_samples, test_max = weighted_ml_opt(np.copy(X_train), oracles, ground_truth, vae_0, **kwargs)
            np.save('results/%s_traj%s.npy' %(test_name, suffix), test_traj)
            np.save('results/%s_oracle_samples%s.npy' % (test_name, suffix), test_oracle_samples)
            np.save('results/%s_gt_samples%s.npy'%(test_name, suffix), test_gt_samples )
            np.save('results/%s_edit_distance_samples%s.npy'%(test_name, suffix), test_edit_distance_samples )

            with open('results/%s_max%s.json'% (test_name, suffix), 'w') as outfile:
                json.dump(test_max, outfile)
        

In [10]:
train_experimental_oracles()
train_experimental_vaes()

Train on 4500 samples, validate on 500 samples
Epoch 1/100
 - 1s - loss: -4.5777e-01 - val_loss: -1.5170e+00
Epoch 2/100
 - 1s - loss: -1.4383e+00 - val_loss: -1.7429e+00
Epoch 3/100
 - 1s - loss: -1.5715e+00 - val_loss: -5.8094e-01
Epoch 4/100
 - 1s - loss: -1.6373e+00 - val_loss: -1.4607e+00
Epoch 5/100
 - 1s - loss: -1.6877e+00 - val_loss: -1.5572e+00
Epoch 6/100
 - 1s - loss: -1.7865e+00 - val_loss: -2.2239e-01
Epoch 7/100
 - 1s - loss: -1.7626e+00 - val_loss: -1.9531e+00
Epoch 8/100
 - 1s - loss: -1.7962e+00 - val_loss: -1.4988e+00
Epoch 9/100
 - 1s - loss: -1.8012e+00 - val_loss: -2.0821e-01
Epoch 10/100
 - 1s - loss: -1.8349e+00 - val_loss: -1.7050e+00
Epoch 11/100
 - 1s - loss: -1.8463e+00 - val_loss: -2.0115e+00
Epoch 12/100
 - 1s - loss: -1.8502e+00 - val_loss: -2.0291e+00
Epoch 13/100
 - 1s - loss: -1.8616e+00 - val_loss: -2.0864e+00
Epoch 14/100
 - 1s - loss: -1.9007e+00 - val_loss: -1.8385e+00
Epoch 15/100
 - 1s - loss: -1.9306e+00 - val_loss: -2.0710e+00
Epoch 16/100
 - 1

 - 1s - loss: -1.9497e+00 - val_loss: -2.0282e+00
Epoch 16/100
 - 1s - loss: -1.9487e+00 - val_loss: -1.5642e+00
Epoch 17/100
 - 1s - loss: -1.9467e+00 - val_loss: -1.6009e+00
Epoch 18/100
 - 1s - loss: -1.9268e+00 - val_loss: -1.9727e+00
Epoch 19/100
 - 1s - loss: -2.0070e+00 - val_loss: -2.0544e+00
Epoch 20/100
 - 1s - loss: -1.9372e+00 - val_loss: -2.0061e+00
Epoch 21/100
 - 1s - loss: -1.9997e+00 - val_loss: -2.0782e+00
Epoch 22/100
 - 1s - loss: -2.0124e+00 - val_loss: -2.0280e+00
Epoch 23/100
 - 1s - loss: -2.0052e+00 - val_loss: -2.0902e+00
Epoch 24/100
 - 1s - loss: -1.9795e+00 - val_loss: -1.6478e+00
Epoch 25/100
 - 1s - loss: -2.0035e+00 - val_loss: -2.0859e+00
Epoch 26/100
 - 1s - loss: -2.0179e+00 - val_loss: -2.0211e+00
Epoch 27/100
 - 1s - loss: -2.0132e+00 - val_loss: -1.9913e+00
Epoch 28/100
 - 1s - loss: -2.1098e+00 - val_loss: -1.5389e+00
Epoch 00028: early stopping
Train on 4500 samples, validate on 500 samples
Epoch 1/100
 - 1s - loss: -6.2695e-01 - val_loss: -1.639

Epoch 2/100
 - 1s - loss: -1.5584e+00 - val_loss: -1.8662e+00
Epoch 3/100
 - 1s - loss: -1.6569e+00 - val_loss: -1.4988e+00
Epoch 4/100
 - 1s - loss: -1.7570e+00 - val_loss: -1.8483e+00
Epoch 5/100
 - 1s - loss: -1.8108e+00 - val_loss: -1.8469e+00
Epoch 6/100
 - 1s - loss: -1.7913e+00 - val_loss: -1.8058e+00
Epoch 7/100
 - 1s - loss: -1.8667e+00 - val_loss: -1.8187e+00
Epoch 00007: early stopping
Train on 4500 samples, validate on 500 samples
Epoch 1/100
 - 1s - loss: -6.7678e-01 - val_loss: -1.4487e+00
Epoch 2/100
 - 1s - loss: -1.2206e+00 - val_loss: -1.3067e+00
Epoch 3/100
 - 1s - loss: -1.4843e+00 - val_loss: -1.6662e+00
Epoch 4/100
 - 1s - loss: -1.6060e+00 - val_loss: -1.8036e+00
Epoch 5/100
 - 1s - loss: -1.5937e+00 - val_loss: -1.4416e+00
Epoch 6/100
 - 1s - loss: -1.5973e+00 - val_loss: -1.8168e+00
Epoch 7/100
 - 1s - loss: -1.6968e+00 - val_loss: -1.8246e+00
Epoch 8/100
 - 1s - loss: -1.7588e+00 - val_loss: -1.8980e+00
Epoch 9/100
 - 1s - loss: -1.8069e+00 - val_loss: -1.9310

Epoch 15/100
 - 1s - loss: -2.0675e+00 - val_loss: -1.7638e+00
Epoch 16/100
 - 1s - loss: -2.0870e+00 - val_loss: -1.8843e+00
Epoch 17/100
 - 1s - loss: -2.0870e+00 - val_loss: -2.0649e+00
Epoch 18/100
 - 1s - loss: -2.1332e+00 - val_loss: -2.1097e+00
Epoch 19/100
 - 1s - loss: -2.1136e+00 - val_loss: -2.0427e+00
Epoch 20/100
 - 1s - loss: -2.1390e+00 - val_loss: -2.1203e+00
Epoch 21/100
 - 1s - loss: -2.1398e+00 - val_loss: -2.0884e+00
Epoch 22/100
 - 1s - loss: -2.1465e+00 - val_loss: -1.7423e+00
Epoch 23/100
 - 1s - loss: -2.1790e+00 - val_loss: -2.0504e+00
Epoch 24/100
 - 1s - loss: -2.1751e+00 - val_loss: -2.0130e+00
Epoch 25/100
 - 1s - loss: -2.1770e+00 - val_loss: -2.0883e+00
Epoch 00025: early stopping
Train on 4500 samples, validate on 500 samples
Epoch 1/100
 - 1s - loss: -8.3797e-01 - val_loss: -1.5368e+00
Epoch 2/100
 - 1s - loss: -1.5912e+00 - val_loss: -1.4192e+00
Epoch 3/100
 - 1s - loss: -1.7174e+00 - val_loss: -1.7546e+00
Epoch 4/100
 - 1s - loss: -1.7733e+00 - val_lo

In [ ]:
run_experimental_weighted_ml(0, repeat_start=0, repeats=3)


In [ ]:
run_experimental_weighted_ml(1, repeat_start=0, repeats=3)


In [ ]:
run_experimental_weighted_ml(2, repeat_start=0, repeats=3)
